In [1]:
import pandas as pd
import os
import numpy as np

### Load the data

In [2]:
csvs = os.listdir('data/accepted/')

In [3]:
# make sure all csv files have the same headers
with open("data/accepted/"+csvs[0], 'r') as f:
    columns = f.readlines()[0]
for c in csvs[1:]:
    with open("data/accepted/" + c, 'r') as f:
        if f.readlines()[0] != columns:
            print(c)

In [2]:
dtype = {'id':str,'member_id':str,'loan_amnt':float,'funded_amnt':float,'funded_amnt_inv':float,
         'term':str,'int_rate':str,
         'installment':float,'grade':str,'sub_grade':str,'emp_title':str,'emp_length':str,'home_ownership':str,
         'annual_inc':float,'verification_status':str,'issue_d':str,'loan_status':str,'pymnt_plan':str,'url':str,
         'desc':str,'purpose':str,'title':str,'zip_code':str,'addr_state':str,'dti':float,'delinq_2yrs':float,
         'earliest_cr_line':str,'inq_last_6mths':float,'mths_since_last_delinq':float,
         'mths_since_last_record':float,
         'open_acc':float,'pub_rec':float,'revol_bal':float,'total_acc':float,'initial_list_status':str,
         'out_prncp':float,
         'out_prncp_inv':float,'total_pymnt':float,'total_pymnt_inv':float,'total_rec_prncp':float,
         'total_rec_int':float,
         'total_rec_late_fee':float,'recoveries':float,'collection_recovery_fee':float,'last_pymnt_d':str,
         'last_pymnt_amnt':float,'next_pymnt_d':str,'last_credit_pull_d':str,'collections_12_mths_ex_med':float,
         'mths_since_last_major_derog':float,'policy_code':str,'application_type':str,'annual_inc_joint':float,
         'dti_joint':float,'verification_status_joint':str,'acc_now_delinq':float,'tot_coll_amt':float,
         'tot_cur_bal':float,
         'open_acc_6m':float,'open_act_il':float,'open_il_12m':float,'open_il_24m':float,
         'mths_since_rcnt_il':float,'total_bal_il':float,
         'il_util':str,'open_rv_12m':float,'open_rv_24m':float,'max_bal_bc':float,'all_util':float,
         'total_rev_hi_lim':float,
         'inq_fi':float,'total_cu_tl':float,'inq_last_12m':float,'acc_open_past_24mths':float,'avg_cur_bal':float,
         'bc_open_to_buy':str,'bc_util':str,'chargeoff_within_12_mths':float,'delinq_amnt':float,
         'mo_sin_old_il_acct':str,
         'mo_sin_old_rev_tl_op':str,'mo_sin_rcnt_rev_tl_op':str,'mo_sin_rcnt_tl':str,'mort_acc':float,
         'mths_since_recent_bc':float,'mths_since_recent_bc_dlq':float,'mths_since_recent_inq':float,
         'mths_since_recent_revol_delinq':float,
         'num_accts_ever_120_pd':float,'num_actv_bc_tl':float,'num_actv_rev_tl':float,'num_bc_sats':float,
         'num_bc_tl':float,
         'num_il_tl':float,'num_op_rev_tl':float,'num_rev_accts':float,'num_rev_tl_bal_gt_0':float,'num_sats':float,
         'num_tl_120dpd_2m':float,'num_tl_30dpd':float,'num_tl_90g_dpd_24m':float,'num_tl_op_past_12m':float,
         'pct_tl_nvr_dlq':float,'percent_bc_gt_75':float,'pub_rec_bankruptcies':float,'tax_liens':float,
         'tot_hi_cred_lim':float,
         'total_bal_ex_mort':float,'total_bc_limit':float,'total_il_high_credit_limit':float,'revol_bal_joint':float,
         'sec_app_earliest_cr_line':str,'sec_app_inq_last_6mths':float,'sec_app_mort_acc':str,'sec_app_open_acc':str,
         'sec_app_revol_util':str,'sec_app_open_act_il':str,'sec_app_num_rev_accts':float,
         'sec_app_chargeoff_within_12_mths':float,
         'sec_app_collections_12_mths_ex_med':float,'sec_app_mths_since_last_major_derog':float,'hardship_flag':str,
         'hardship_type':str,'hardship_reason':str,'hardship_status':str,'deferral_term':str,'hardship_amount':float,
         'hardship_start_date':str,'hardship_end_date':str,'payment_plan_start_date':str,'hardship_length':str,
         'hardship_dpd':float,'hardship_loan_status':str,'orig_projected_additional_accrued_interest':float,
         'hardship_payoff_balance_amount':float,'hardship_last_payment_amount':float,'disbursement_method':str,
         'debt_settlement_flag':str,'debt_settlement_flag_date':str,'settlement_status':str,'settlement_date':str,
         'settlement_amount':float,'settlement_percentage':float,'settlement_term':str}

In [5]:
df = pd.DataFrame(columns=dtype.keys())
for i,c in enumerate(csvs):
    df = df.append(pd.read_csv("data/accepted/" + c, dtype=dtype))

In [6]:
print("number of data points: {}".format(len(df)))

number of data points: 1646801


### Drop current and unlabeled borrowers

In [7]:
print(df.loan_status.unique())

['Fully Paid' 'Charged Off' nan
 'Does not meet the credit policy. Status:Fully Paid'
 'Does not meet the credit policy. Status:Charged Off' 'Current'
 'Late (31-120 days)' 'In Grace Period' 'Late (16-30 days)' 'Default']


In [8]:
len(df[df.loan_status=='Current'].index)

810381

In [9]:
len(df[df.loan_status!='Current'].index)

836420

In [10]:
# drop current borrowers
df = df[df.loan_status!='Current']
len(df)

836420

In [11]:
print(df.loan_status.unique())

['Fully Paid' 'Charged Off' nan
 'Does not meet the credit policy. Status:Fully Paid'
 'Does not meet the credit policy. Status:Charged Off'
 'Late (31-120 days)' 'In Grace Period' 'Late (16-30 days)' 'Default']


We drop current borrowers, as we are primarily interested in those who have already paid off or charged off on their loans. We save the current borrowers to predict how they will behave for future projects. Note that "Charged Off" means no further payment is expected. This typically occurs when a payment is 150+ days past due. "Default" typically means the borrower has not made a payment for 121+ days, but the account has not been closed or sent to collections yet (see https://help.lendingclub.com/hc/en-us/articles/216127747-What-is-the-difference-between-a-loan-that-is-in-default-and-a-loan-that-has-been-charged-off-)

In [12]:
current_df = df[(df.loan_status != 'Fully Paid') & (df.loan_status != 'Charged Off') 
                & (df.loan_status != 'Does not meet the credit policy. Status:Fully Paid')
                & (df.loan_status != 'Does not meet the credit policy. Status:Charged Off')]

In [13]:
current_df.to_csv("data/current_borrowers.csv", index=False)

In [14]:
# drop current borrowers
df = df[df.loan_status!='In Grace Period']
df = df[df.loan_status!='Late (31-120 days)']
df = df[df.loan_status!='Late (16-30 days)']
df = df[df.loan_status!='Default']
len(df)

790682

We drop the unlabeled data points, as the proportion of unlabeled points (about .003%) is marginal.

In [15]:
print("number of unlabeled points: {}".format(len(df) - df.loan_status.value_counts().sum()))

number of unlabeled points: 23


In [16]:
# drop unlabeled data points (nan)
df = df.dropna(subset=['loan_status'])
len(df)

790659

In [17]:
print(df.loan_status.unique())

['Fully Paid' 'Charged Off'
 'Does not meet the credit policy. Status:Fully Paid'
 'Does not meet the credit policy. Status:Charged Off']


In [18]:
df.loan_status.value_counts()

Fully Paid                                             627124
Charged Off                                            160786
Does not meet the credit policy. Status:Fully Paid       1988
Does not meet the credit policy. Status:Charged Off       761
Name: loan_status, dtype: int64

Observations labeled with "Does not meet the credit policy" are people that borrowed from Lending Club in the past, but would not currently qualify for a loan under Lending Club's current policy. We save them as a separate dataframe for further analysis; perhaps we can identify the change in Lending Club's policy that would cause those borrowers to not qualify for a loan today.

In [19]:
unqualified_df = df[df.loan_status.str.contains("Does not meet the credit policy")]
len(unqualified_df)

2749

In [20]:
unqualified_df.to_csv("data/unqualified_df.csv", index=False)
df = df[(df.loan_status != 'Does not meet the credit policy. Status:Fully Paid') & 
        (df.loan_status != 'Does not meet the credit policy. Status:Charged Off')]

In [21]:
len(df)

787910

### Label the data

In [22]:
df.loan_status = df.loan_status.map({'Fully Paid':1, 'Charged Off':0})

In [24]:
df.loan_status.value_counts()

1    627124
0    160786
Name: loan_status, dtype: int64

In [25]:
df.to_csv('data/labeled.csv', index=False)

### Get rid of unique/uniform columns

In [3]:
dtype['loan_status'] = int
df = pd.read_csv('data/labeled.csv', dtype=dtype)
len(df.columns)

145

In [4]:
# find features with uniform values
rows = len(df)
for col in df.columns:
    if len(df[col].unique()) == 1:
        print(col, len(df[col].unique()))
        df = df.drop(col, axis=1)

hardship_flag 1
id 1
member_id 1
next_pymnt_d 1
out_prncp 1
out_prncp_inv 1
policy_code 1
pymnt_plan 1
url 1


In [8]:
print(len(df.columns))
print(df.shape)

136
(787910, 136)


Now we check for null values. For columns with a large amount of missing values (70% or more), we drop them.

In [9]:
null_counts = dict(df.isnull().sum())
null_cols = []
for key,val in null_counts.items():
    if val != 0:
        print(key, '\t', val/df.shape[0])
    if val/df.shape[0] >= 0.7:
        null_cols.append(key)

acc_open_past_24mths 	 0.060008122755137006
all_util 	 0.7942696500869388
annual_inc_joint 	 0.9958002817580688
avg_cur_bal 	 0.08571918112474775
bc_open_to_buy 	 0.06964754857788326
bc_util 	 0.07024025586678681
chargeoff_within_12_mths 	 7.10741074488203e-05
collections_12_mths_ex_med 	 7.10741074488203e-05
debt_settlement_flag_date 	 0.9854691525681867
deferral_term 	 0.9994567907502125
desc 	 0.8483748143823533
dti 	 3.6806234214567655e-05
dti_joint 	 0.9958002817580688
emp_title 	 0.05929484331966849
hardship_amount 	 0.9994567907502125
hardship_dpd 	 0.9994567907502125
hardship_end_date 	 0.9994567907502125
hardship_last_payment_amount 	 0.9994567907502125
hardship_length 	 0.9994567907502125
hardship_loan_status 	 0.9994567907502125
hardship_payoff_balance_amount 	 0.9994567907502125
hardship_reason 	 0.9994567907502125
hardship_start_date 	 0.9994567907502125
hardship_status 	 0.9994567907502125
hardship_type 	 0.9994567907502125
il_util 	 0.8200835120762524
inq_fi 	 0.79425695

In [10]:
print(len(null_cols))

52


In [11]:
df = df.drop(null_cols, axis=1)
len(df.columns)

84

In [12]:
for col in df.columns:
    if df[col].dtype == 'O':
        print(col)

addr_state
application_type
bc_open_to_buy
bc_util
debt_settlement_flag
disbursement_method
earliest_cr_line
emp_length
emp_title
grade
home_ownership
initial_list_status
int_rate
issue_d
last_credit_pull_d
last_pymnt_d
mo_sin_old_il_acct
mo_sin_old_rev_tl_op
mo_sin_rcnt_rev_tl_op
mo_sin_rcnt_tl
purpose
revol_util
sub_grade
term
title
verification_status


In [5]:
df['zip_code'].head()

0    860xx
1    309xx
2    606xx
3    917xx
4    972xx
Name: zip_code, dtype: object

In [6]:
df.zip_code = df.zip_code.apply(lambda x: int(x[:3]))
df['zip_code'].head()

0    860
1    309
2    606
3    917
4    972
Name: zip_code, dtype: int64

In [7]:
df['term'].head(15)

0      36 months
1      60 months
2      36 months
3      36 months
4      60 months
5      36 months
6      60 months
7      36 months
8      60 months
9      60 months
10     60 months
11     36 months
12     36 months
13     36 months
14     36 months
Name: term, dtype: object

In [8]:
df.term = df.term.apply(lambda x: int(x.split()[0]))
df['term'].head()

0    36
1    60
2    36
3    36
4    60
Name: term, dtype: int64

In [9]:
df['verification_status'].value_counts()

Source Verified    283045
Verified           261725
Not Verified       243140
Name: verification_status, dtype: int64

In [10]:
df['title'].head()

0                Computer
1                    bike
2    real estate business
3                personel
4                Personal
Name: title, dtype: object

In [11]:
df['sub_grade'].head()

0    B2
1    C4
2    C5
3    C1
4    B5
Name: sub_grade, dtype: object

In [12]:
df['revol_util'].head()

0    83.7%
1     9.4%
2    98.5%
3      21%
4    53.9%
Name: revol_util, dtype: object

In [13]:
df.revol_util.str[:-1].astype('float')/100

0         0.8370
1         0.0940
2         0.9850
3         0.2100
4         0.5390
5         0.2830
6         0.8560
7         0.8750
8         0.3260
9         0.3650
10        0.2060
11        0.6710
12        0.9170
13        0.4310
14        0.5550
15        0.8150
16        0.7020
17        0.1600
18        0.3773
19        0.2310
20        0.8560
21        0.9030
22        0.8240
23        0.9180
24        0.2970
25        0.9390
26        0.5760
27        0.5950
28        0.3770
29        0.5910
           ...  
787880    0.6620
787881    0.0420
787882    0.4600
787883    0.4000
787884    0.1630
787885    0.3870
787886    0.4700
787887    0.8150
787888    0.6540
787889    0.6730
787890    0.4940
787891    0.7340
787892    0.2240
787893    0.7030
787894    0.6840
787895    0.0030
787896    0.6900
787897    0.0880
787898    0.2350
787899    0.3770
787900    0.1240
787901    0.4760
787902    0.6250
787903    0.2000
787904    0.4050
787905    0.5830
787906    0.5870
787907    0.66

In [14]:
df.revol_util = df.revol_util.str[:-1].astype('float')/100

In [15]:
df.revol_util.head()

0    0.837
1    0.094
2    0.985
3    0.210
4    0.539
Name: revol_util, dtype: float64

In [16]:
df['purpose'].head() # consider using purpose and dropping title, since title is user input, purpose is set categorical

0       credit_card
1               car
2    small_business
3             other
4             other
Name: purpose, dtype: object

In [17]:
df['mo_sin_rcnt_tl'].head(15)

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
5     NaN
6     NaN
7     NaN
8     NaN
9     NaN
10    NaN
11    NaN
12    NaN
13    NaN
14    NaN
Name: mo_sin_rcnt_tl, dtype: object

In [18]:
df['mo_sin_rcnt_tl'].isnull().sum()/df.shape[0]

0.08570395095886586

In [19]:
df['mo_sin_rcnt_rev_tl_op'].isnull().sum()/df.shape[0]

0.08570522013935601

In [20]:
df['mo_sin_old_rev_tl_op'].isnull().sum()/df.shape[0]

0.08570522013935601

In [21]:
df['mo_sin_old_il_acct'].isnull().sum()/df.shape[0]

0.11267403637471285

In [22]:
df['last_pymnt_d'].head()

0    Jan-2015
1    Apr-2013
2    Jun-2014
3    Jan-2015
4    Jan-2017
Name: last_pymnt_d, dtype: object

In [23]:
df['last_pymnt_d'] = pd.to_datetime(df['last_pymnt_d'])

In [24]:
df['last_pymnt_d'].head()

0   2015-01-01
1   2013-04-01
2   2014-06-01
3   2015-01-01
4   2017-01-01
Name: last_pymnt_d, dtype: datetime64[ns]

In [25]:
df['last_credit_pull_d'] = pd.to_datetime(df['last_credit_pull_d'])

In [26]:
df['last_credit_pull_d'].head()

0   2017-11-01
1   2016-10-01
2   2017-06-01
3   2016-04-01
4   2017-01-01
Name: last_credit_pull_d, dtype: datetime64[ns]

In [27]:
df['issue_d'].head()

0    Dec-2011
1    Dec-2011
2    Dec-2011
3    Dec-2011
4    Dec-2011
Name: issue_d, dtype: object

In [28]:
df['issue_d'] = pd.to_datetime(df['issue_d'])
df['issue_d'].head()

0   2011-12-01
1   2011-12-01
2   2011-12-01
3   2011-12-01
4   2011-12-01
Name: issue_d, dtype: datetime64[ns]

In [29]:
df['int_rate'].head()

0     10.65%
1     15.27%
2     15.96%
3     13.49%
4     12.69%
Name: int_rate, dtype: object

In [30]:
df.int_rate = df.int_rate.str[:-1].astype('float')/100

In [31]:
df.int_rate.head()

0    0.1065
1    0.1527
2    0.1596
3    0.1349
4    0.1269
Name: int_rate, dtype: float64

In [32]:
df.initial_list_status.value_counts()

f    404280
w    383630
Name: initial_list_status, dtype: int64

In [33]:
df.home_ownership.value_counts()

MORTGAGE    393581
RENT        314384
OWN          79670
OTHER          144
ANY             88
NONE            43
Name: home_ownership, dtype: int64

In [34]:
df.grade.value_counts()

B    227626
C    217826
A    129000
D    123656
E     61551
F     22423
G      5828
Name: grade, dtype: int64

In [35]:
df.emp_title.head()

0                         NaN
1                       Ryder
2                         NaN
3         AIR RESOURCES BOARD
4    University Medical Group
Name: emp_title, dtype: object

In [36]:
len(df.emp_title.unique())

274750

In [37]:
df.earliest_cr_line.head()

0    Jan-1985
1    Apr-1999
2    Nov-2001
3    Feb-1996
4    Jan-1996
Name: earliest_cr_line, dtype: object

In [38]:
df.disbursement_method.value_counts()

Cash         786873
DirectPay      1037
Name: disbursement_method, dtype: int64

In [39]:
df.debt_settlement_flag.value_counts()

N    776461
Y     11449
Name: debt_settlement_flag, dtype: int64

In [40]:
df.bc_util = df.bc_util.astype('float')

In [41]:
df.bc_util.tail(15)

787895      0.3
787896    100.1
787897      9.8
787898     24.5
787899     64.7
787900     12.8
787901     51.7
787902     83.5
787903     20.0
787904     54.2
787905     60.5
787906     68.0
787907     61.7
787908     95.8
787909     48.0
Name: bc_util, dtype: float64

In [42]:
df.bc_util.isnull().sum() # I suspect that these are missing for a particular csv subset of the data, same for below

55343

In [43]:
df.bc_open_to_buy = df.bc_open_to_buy.astype('float')

In [44]:
df.bc_open_to_buy.tail(15)

787895    40278.0
787896        0.0
787897    52566.0
787898    53687.0
787899     8710.0
787900    74790.0
787901     2416.0
787902     3582.0
787903    28475.0
787904    12830.0
787905     4625.0
787906    10386.0
787907    18465.0
787908      921.0
787909    13771.0
Name: bc_open_to_buy, dtype: float64

In [45]:
df.application_type.value_counts()

Individual    784601
Joint App       3309
Name: application_type, dtype: int64

In [47]:
len(df.addr_state.unique())

51